In [2]:
import builtins
import torch 
from torch import nn
import numpy as np
import random as rnd
import matplotlib

# Set the backend BEFORE importing pyplot
matplotlib.use('TkAgg')  # Or 'Qt5Agg', 'WXAgg', etc.

import matplotlib.pyplot as plt



In [4]:
from sklearn.datasets import make_circles
n_samples = 1000
X, y = make_circles(n_samples,
                    noise=0.03, # a little bit of noise to the dots
                    random_state=42) # keep random state so we get the same values

X = torch.from_numpy(X).type(torch.float)
y = torch.from_numpy(y).type(torch.float)

device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [132]:
#Vamos criar um conjunto de dados para o modelo classificar
def curva1(t): #circulo
    noise = 0.02
    return([np.cos(np.pi*2*t)/2-0.1+rnd.random()*noise- noise/2, np.sin(np.pi*2*t)/2+0.1+rnd.random()*noise- noise/2])

def curva2(t): #2
    noise = 0.02
    C = rnd.random()
    if C > 0.25:
        return([(3.35*t+1.2)*0.14*np.cos((3.35*t+1.2)+9.6)    +    rnd.random()*noise- noise/2,     -(3.35*t+1.2)*0.146*np.sin((3.35*t+1.2)+9.6)    +    rnd.random()*noise- noise/2])
    else:
        return([ -0.01+0.41*t + rnd.random()*noise- noise/2,  -0.666 + rnd.random()*noise- noise/2])


        
def curva3(t): #2
    noise = 0.02
    k =0.04
    j = 0.59
    v = 1.1
    n = 0.984
    m = -0.33
    l = -1.04
    
    C = rnd.random()
    if C > 0.5: 
        return([v*(-1.155 + 1.415*t)**2+n*(-1.155 + 1.415*t)+m    +    rnd.random()*noise- noise/2,     l*(-1.155 + 1.415*t)**2+k*(-1.155 + 1.415*t)+j    +    rnd.random()*noise- noise/2])
    else:
        return([-(v*(-1.155 + 1.415*t)**2+n*(-1.155 + 1.415*t)+m)    +    rnd.random()*noise- noise/2,     l*(-1.155 + 1.415*t)**2+k*(-1.155 + 1.415*t)+j    +    rnd.random()*noise- noise/2])

def plano_de_fundo():
    return(2*rnd.random()-1, 2*rnd.random()-1)


Lx1 = []
Ly1 = []
Lx2 = []
Ly2 = []
Lx3 = []
Ly3 = []
LxF = []
LyF = []

N_Dados_Por_Curva = 2000
for i in range(N_Dados_Por_Curva):
    a = curva1(rnd.random())
    b = curva2(rnd.random())
    c = curva3(rnd.random())
    Lx1 = Lx1 + [a[0]]
    Ly1 = Ly1 + [a[1]]
    Lx2 = Lx2 + [b[0]]
    Ly2 = Ly2 + [b[1]]
    Lx3 = Lx3 + [c[0]]
    Ly3 = Ly3 + [c[1]]
    for j in range(3):
        d = plano_de_fundo()
        LxF = LxF + [d[0]]
        LyF = LyF + [d[1]]


Lista_Dados_X_Transposta = [Lx1 + Lx2 + Lx3 + LxF, Ly1 + Ly2 + Ly3+ LyF]
Dados_X_Transpostos = torch.tensor(Lista_Dados_X_Transposta)
Dados_X = Dados_X_Transpostos.T

Lista_Dados_Y = []
for j in range(6):
    for i in range(N_Dados_Por_Curva):
        if j == 0:
            Lista_Dados_Y = Lista_Dados_Y + [[1, 0, 0, 0]]
        elif j == 1:
            Lista_Dados_Y = Lista_Dados_Y + [[0, 1, 0, 0]]
        elif j == 2:
            Lista_Dados_Y = Lista_Dados_Y + [[0, 0, 1, 0]]
        else:
            Lista_Dados_Y = Lista_Dados_Y + [[0, 0, 0, 1]]

Dados_Y = torch.tensor(Lista_Dados_Y)

Dados_X = Dados_X.to(device)
Dados_Y = Dados_Y.to(device)

plt.scatter(Lx1, Ly1, s = 1, c = 'red')
plt.scatter(Lx2, Ly2, s = 1, c = 'blue')
plt.scatter(Lx3, Ly3, s = 1, c = 'green')
plt.scatter(LxF, LyF, s = 1, c = 'orange')
plt.show()

In [ ]:
class CircleModelV0(nn.Module):
    def __init__(self):
        super().__init__()
        # 2. Create 2 nn.Linear layers capable of handling X and y input and output shapes
        self.layer_1 = nn.Linear(in_features=2, out_features=20) # takes in 2 features (X), produces 5 features
        self.layer_2 = nn.Linear(in_features=20, out_features=1) # takes in 5 features, produces 1 feature (y)
        self.relu = nn.ReLU()
    # 3. Define a forward method containing the forward pass computation
    def forward(self, x):
        # Return the output of layer_2, a single feature, the same shape as y
        return torch.sigmoid(self.layer_2(self.relu(self.layer_1(x)))) # computation goes through layer_1 first then the output of layer_1 goes through layer_2

# 4. Create an instance of the model and send it to target device
model_0 = CircleModelV0().to(device)
model_0

loss_function = nn.BCEWithLogitsLoss()
#loss_function = torch.nn.L1Loss()
optimizer = torch.optim.SGD(params = model_0.parameters(), lr=1)

#torch.manual_seed(22)

epochs = 10000

X, y = X.to(device), y.to(device)
#print(len(y))
for epoch in builtins.range(epochs):
    ### Training 
    # 1. Forward pass
    
    y_pred = model_0(X)
    y_pred = y_pred.squeeze()
    
    #print(y_pred, y)
    
    # 2. Calculate loss (no accuracy since it's a regression problem, not classification)
    loss = loss_function(y_pred, y)


    # 3. Optimizer zero grad
    optimizer.zero_grad()

    # 4. Loss backwards
    loss.backward()

    # 5. Optimizer step
    optimizer.step()

    '''
  
            
    if epoch % 300 == 0:
        accuracy = 0
        #print(loss)
        
        for i in builtins.range(len(y)):
            if torch.round(y_pred)[i]==y[i]:
                accuracy = accuracy + 1/len(y)
        print('accuracy:', accuracy)
    
    if epoch % 3000 == 0:
        print(torch.round(y_pred) == y)
    '''


        
    '''
    ### Testing
    model_2.eval()
    with torch.inference_mode():
      # 1. Forward pass
      test_pred = model_2(X_test_regression)
      # 2. Calculate the loss 
      test_loss = loss_fn(test_pred, y_test_regression)

    # Print out what's happening
    if epoch % 100 == 0: 
        print(f"Epoch: {epoch} | Train loss: {loss:.5f}, Test loss: {test_loss:.5f}")'''



In [38]:
R = 30

Lx = []
for i in builtins.range(-R, R):
    for j in builtins.range(-R, R):
        Lx = Lx + [[i/R, j/R]]

Px = torch.tensor(Lx)
Px = Px.to(device)
Py_pred = model_0(Px)
Py_pred = torch.round(Py_pred)

Px = Px.to("cpu")
Py_pred.to("cpu")
#print(Px, Py_pred)

Px = Px.squeeze()
Py_pred = Py_pred.squeeze()
#print(Px, Py_pred)

LXAzul = []
LYAzul = []
LXVermelho = []
LYVermelho = []

for i in builtins.range(len(Px)):
    if Py_pred[i] == 0:
        LXAzul = LXAzul + [Px[i][0]]
        LYAzul = LYAzul + [Px[i][1]]
    else:
        LXVermelho = LXVermelho + [Px[i][0]]
        LYVermelho = LYVermelho + [Px[i][1]]
plt.scatter(LXAzul, LYAzul, c="mediumblue")
plt.scatter(LXVermelho, LYVermelho, c="red")

LXVerde =[]
LYVerde = []
LXAmarelo = []
LYAmarelo = []

X1, y1 = X.to("cpu"), y.to("cpu")
for i in builtins.range(len(X1)):
    if y1[i] == 0:
        LXVerde = LXVerde + [X1[i][0]]
        LYVerde = LYVerde + [X1[i][1]]
    else:
        LXAmarelo = LXAmarelo + [X1[i][0]]
        LYAmarelo = LYAmarelo + [X1[i][1]]
        

plt.scatter(LXVerde, LYVerde, c="green")
plt.scatter(LXAmarelo, LYAmarelo, c="yellow")

plt.show()

NameError: name 'model_0' is not defined

In [130]:
#def Softmax():
    

class C3_Model(nn.Module):
    def __init__(self):
        super().__init__()
        # 2. Create 2 nn.Linear layers capable of handling X and y input and output shapes
        self.layer_1 = nn.Linear(in_features=2, out_features=10) # takes in 2 features (X), produces 5 features
        self.layer_2 = nn.Linear(in_features=10, out_features=10) # takes in 5 features, produces 1 feature (y)
        self.layer_3 = nn.Linear(in_features=10, out_features=4)
        self.relu = nn.ReLU()
    # 3. Define a forward method containing the forward pass computation
    def forward(self, x):
        # Return the output of layer_2, a single feature, the same shape as y
        return self.layer_3(self.relu(self.layer_2(self.relu(self.layer_1(x))))) # computation goes through layer_1 first then the output of layer_1 goes through layer_2

# 4. Create an instance of the model and send it to target device
model_1 = C3_Model().to(device)
model_1


loss_function = nn.MSELoss()
#loss_function = torch.nn.L1Loss()
optimizer = torch.optim.SGD(params = model_1.parameters(), lr = 1)

#torch.manual_seed(22)

epochs = 20000

#print(len(y))
for epoch in builtins.range(epochs):
    ### Training 
    # 1. Forward pass
    
    logis = model_1(Dados_X.float())
    y_pred = torch.softmax(logis, dim = 1)
    y_pred = y_pred.squeeze()
    
    #print(logis, y_pred, Dados_Y)
    
    # 2. Calculate loss (no accuracy since it's a regression problem, not classification)
    loss = loss_function(y_pred, Dados_Y.float())
    Nloss = loss.item()
    #Nloss = np.float16(Nloss)
    # 3. Optimizer zero grad
    optimizer.zero_grad()

    for group in optimizer.param_groups:
        group['lr'] = (40*Nloss)

    
    # 4. Loss backwards
    loss.backward()

    # 5. Optimizer step
    optimizer.step()

    
    if loss < 0.01:
        print(loss, epoch)
        
        break
            
    if epoch % 1000 == 0:
        accuracy = 0
        print(loss)
    
    if epoch % 10000 ==0:    
        for i in range(len(Dados_X)):
            if torch.argmax(y_pred[i]) == torch.argmax(Dados_Y[i]):
                accuracy = accuracy + 1/len(Dados_X)
        print(accuracy)

        '''
        for i in builtins.range(len(y)):
            if torch.round(y_pred)[i]==y[i]:
                accuracy = accuracy + 1/len(y)
        print('accuracy:', accuracy)
        
    if epoch % 3000 == 0:
        print(torch.round(y_pred) == y)
    '''

for i in range(len(Dados_X)):
    if torch.argmax(y_pred[i]) == torch.argmax(Dados_Y[i]):
        accuracy = accuracy + 1/len(Dados_X)
print(accuracy)

tensor(0.2010, device='cuda:0', grad_fn=<MseLossBackward0>)
0.1165000000000024
tensor(0.0743, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(0.0685, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(0.0623, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(0.0591, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(0.0526, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(0.0489, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(0.0473, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(0.0484, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(0.0447, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(0.0407, device='cuda:0', grad_fn=<MseLossBackward0>)
0.9013333333332507
tensor(0.0437, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(0.0420, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(0.0410, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(0.0402, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(0.0397, device='cuda:0', grad_fn=<MseLossBackward0>)
te

In [133]:
R = 100

Lx = []
for i in builtins.range(-R, R):
    for j in builtins.range(-R, R):
        Lx = Lx + [[i/R, j/R]]

Px = torch.tensor(Lx)
Px = Px.to(device)
Py_pred = model_1(Px)


Px = Px.to("cpu")
Py_pred = Py_pred.to("cpu")
#print(Px, Py_pred)

Px = Px.squeeze()
Py_pred = Py_pred.squeeze()
#print(Px, Py_pred)

Px = Px.tolist()
Py_pred = Py_pred.tolist()

LXAmarelo = []
LYAmarelo = []
LXRoxo = []
LYRoxo = []
LXPreto = []
LYPreto = []
LXFundo = []
LYFundo = []


for i in builtins.range(len(Px)):
    if np.argmax(Py_pred[i]) == 0:
        LXAmarelo = LXAmarelo + [Px[i][0]]
        LYAmarelo = LYAmarelo + [Px[i][1]]
    elif np.argmax(Py_pred[i]) == 1:
        LXRoxo = LXRoxo + [Px[i][0]]
        LYRoxo = LYRoxo + [Px[i][1]]
    elif np.argmax(Py_pred[i]) == 2:
        LXPreto = LXPreto + [Px[i][0]]
        LYPreto = LYPreto + [Px[i][1]]
    else:
        LXFundo = LXFundo + [Px[i][0]]
        LYFundo = LYFundo + [Px[i][1]]

S1 = 5        
plt.scatter(LXAmarelo, LYAmarelo, c="yellow", s = S1)
plt.scatter(LXRoxo, LYRoxo, c="purple", s = S1)
plt.scatter(LXPreto, LYPreto, c="black", s = S1)
plt.scatter(LXFundo, LYFundo, c="orange", s = S1)




plt.scatter(Lx1, Ly1, s = 2, c = 'red')
plt.scatter(Lx2, Ly2, s = 2, c = 'blue')
plt.scatter(Lx3, Ly3, s = 2, c = 'green')
plt.show()